In [ ]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import os

from tqdm import tqdm
import cohere
import pandas as pd
from pandas import DataFrame
import numpy as np
import warnings

from IPython.display import display

warnings.filterwarnings("ignore")

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


API keys:

In [319]:
from dotenv import load_dotenv
load_dotenv(override=True)

# Replace with your own PineCone API KEY
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY", "") 

# Replace with your own Cohere API KEY
COHERE_API_KEY = os.environ.get("COHERE_API_KEY", "")

In [128]:
kb_path = '/Users/ransela/Desktop/data_science_degree/4th_year/spring/Data Analysis and Visualization Lab/project/Whatsapp_webApp_-Django-/RAG/RAG_data/KB_data.csv'
whatsapp_chats = pd.read_csv(kb_path)

In [129]:
whatsapp_chats.head()

,msg_id,conv_id,conv_turn,sender_user_id,receiver_user_id,sent_at,text,answer
0,37135e2a-f6c5-482c-825b-362e797caf42,chat:u_barbara_u_maayan,1,u_barbara,u_maayan,2025-01-13 12:49:06,"Hi, sweetheart.\nTell me, do you happen to hav...",I broke up with my girlfriend today
1,2db1ff10-fe22-4e4a-aae5-f23947177c26,chat:u_barbara_u_maayan,2,u_maayan,u_barbara,2025-01-13 21:25:20,I broke up with my girlfriend today,"By any chance, could you replace me face-to-fa..."
2,5cc8ce01-f038-46d4-aa11-2a63650e79ee,chat:u_barbara_u_maayan,3,u_barbara,u_maayan,2025-01-13 21:25:20,"By any chance, could you replace me face-to-fa...",That's the excuse\nUmmmmm\nDoubt it
3,34523a17-26eb-4f26-b0b4-342645dfa5d0,chat:u_barbara_u_maayan,4,u_maayan,u_barbara,2025-01-13 21:25:23,That's the excuse\nUmmmmm\nDoubt it,"Oh no, sorryyyyy"
4,b49c88d0-bb97-4d37-8e3e-40fd90943854,chat:u_barbara_u_maayan,5,u_barbara,u_maayan,2025-01-13 21:25:31,"Oh no, sorryyyyy",In Karmiel?


messages sent to our user_id (u_barbara)

In [130]:
messages_sent = whatsapp_chats[whatsapp_chats['receiver_user_id'] == 'u_barbara'].sort_values(by='conv_turn')

## **Preprocessing & Embedding the data**

In [131]:
def load_and_embedd_dataset(
        dataset: DataFrame,
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
) -> DataFrame:
    """
    Return a dataset with  column of embedd text field using a sentence-transformer model
    Args:
        dataset_name: The name of the dataset to load
        model: The model to use for embedding
    Returns:
        tuple: A Dataset containing the new column of the the embeddings
    """

    print("Loading and embedding the dataset")

    # build the text we embed
    dataset["doc_text"] = (
        dataset["sender_user_id"] + ": " + dataset["text"].fillna("") + "\n" +
        dataset["receiver_user_id"] + ": " + dataset["answer"].fillna("")
    )

       
    

    # compute embeddings as numpy array [n_rows, dim]
    embeddings = model.encode(
        dataset["doc_text"].tolist(),
        batch_size=32,
        convert_to_numpy=True,
        show_progress_bar=True
    )

    dataset["embedding"] = list(embeddings)

    print("Done!")
    return dataset, embeddings

In [133]:
from sentence_transformers import SentenceTransformer
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
model_emb = SentenceTransformer(EMBEDDING_MODEL)

In [134]:
kb_df,embeddings = load_and_embedd_dataset(messages_sent, model_emb)
kb_df.head()

Loading and embedding the dataset


Batches: 100%|██████████| 9/9 [00:01<00:00,  5.46it/s]

Done!


,msg_id,conv_id,conv_turn,sender_user_id,receiver_user_id,sent_at,text,answer,doc_text,embedding
1,2db1ff10-fe22-4e4a-aae5-f23947177c26,chat:u_barbara_u_maayan,2,u_maayan,u_barbara,2025-01-13 21:25:20,I broke up with my girlfriend today,"By any chance, could you replace me face-to-fa...",u_maayan: I broke up with my girlfriend today\...,"[-0.106244296, 0.017279046, 0.08007642, 0.0052..."
3,34523a17-26eb-4f26-b0b4-342645dfa5d0,chat:u_barbara_u_maayan,4,u_maayan,u_barbara,2025-01-13 21:25:23,That's the excuse\nUmmmmm\nDoubt it,"Oh no, sorryyyyy",u_maayan: That's the excuse\nUmmmmm\nDoubt it\...,"[-0.009083027, 0.020246388, -0.0037002887, -0...."
5,6502b89a-e345-4fea-9577-8fc4ecf72772,chat:u_barbara_u_maayan,6,u_maayan,u_barbara,2025-01-13 21:25:33,In Karmiel?,No,u_maayan: In Karmiel?\nu_barbara: No,"[-0.048872855, 0.029274564, -0.049886603, 0.02..."
7,6b4fb49c-c428-44da-ba68-3e05cf1591a0,chat:u_barbara_u_maayan,8,u_maayan,u_barbara,2025-01-13 21:25:36,Ask Itay,In Haifa\nFor 10th grade (Yud),u_maayan: Ask Itay\nu_barbara: In Haifa\nFor 1...,"[0.020345852, 0.07910001, -0.04930794, -0.0369..."
9,498fea35-9088-45a1-8358-0215e41cc717,chat:u_barbara_u_maayan,10,u_maayan,u_barbara,2025-01-13 21:25:40,Ah in Haifa\nPossible then\nAlthough I don't know,Keep me updated,u_maayan: Ah in Haifa\nPossible then\nAlthough...,"[0.010905846, 0.018512694, -0.046605963, -0.02..."


In [135]:
shape = embeddings.shape
print("Embedding shape:", shape)

Embedding shape: (258, 384)


### Create Context and User_style examples

Creating a context window for desired conversation

In [136]:
def build_context(
    df: pd.DataFrame,
    conv_id: str,
    k: int = 10,
    text_col: str = "text",
) -> str:
    """
    Return a string with the last k turns from a given conversation.

    Each line looks like:
        <sender_user_id>: <message>

    Args:
        df: DataFrame with at least ['conv_id', 'sender_user_id', text_col, 'sent_at'].
        conv_id: The conversation id to extract from.
        k: Number of turns to include (from the end of the conversation).
        text_col: Column name that holds the actual message text.

    Returns:
        A single multi line string that can be dropped straight into the prompt.
    """
    conv_df = (
        df[df["conv_id"] == conv_id]
        .sort_values("sent_at")          # or 'conv_turn' if you prefer
        .tail(k)
    )

    lines = [
        f"{row['sender_user_id']}: {row[text_col]}"
        for _, row in conv_df.iterrows()
    ]

    context = "\n".join(lines)
    return context

Creating a User's Style (sample of k mesages)

In [137]:

def build_user_style(
    df: pd.DataFrame,
    user_id: str,
    k: int = 10,
    text_col: str = "text",
    random_sample: bool = True,
    seed: int | None = 42,
) -> str:
    """
    Return a string that represents the typical style of a given user,
    built from k of their messages.

    Each line looks like:
        <message>

    Args:
        df: DataFrame with at least ['sender_user_id', text_col].
        user_id: The user whose style we want to capture.
        k: Number of messages to use.
        text_col: Column with the text of the message.
        random_sample: If True sample k messages randomly, else take the last k.
        seed: Random seed for reproducibility when random_sample is True.

    Returns:
        A single multi line string with example messages in the user's style.
    """
    user_df = df[df["sender_user_id"] == user_id].copy()

    if len(user_df) == 0:
        return ""

    user_df = user_df.sort_values("sent_at")

    if random_sample and len(user_df) > k:
        rng = np.random.default_rng(seed)
        idx = rng.choice(user_df.index.to_list(), size=k, replace=False)
        user_df = user_df.loc[idx].sort_values("sent_at")
    else:
        user_df = user_df.tail(k)

    lines = [str(msg) for msg in user_df[text_col].tolist()]
    user_style = "\n".join(lines)
    return user_style

## **Inserting the data into Pinecone VectorDB**


### creating the index

In [138]:
def create_pinecone_index(
        index_name: str,
        dimension: int,
        metric: str = 'cosine',
):
    """
    Create a pinecone index if it does not exist
    Args:
        index_name: The name of the index
        dimension: The dimension of the index
        metric: The metric to use for the index
    Returns:
        Pinecone: A pinecone object which can later be used for upserting vectors and connecting to VectorDBs
    """
     
    print("Creating a Pinecone index...")
    pc = Pinecone(api_key=PINECONE_API_KEY)
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=dimension,

            metric=metric,
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    print("Done!")
    return pc

In [139]:
INDEX_NAME = 'chats-index'

# Create the vector database
# We are passing the index_name and the size of our embeddings
pc = create_pinecone_index(INDEX_NAME, shape[1])

Creating a Pinecone index...
Done!


inserting data into the index

In [143]:
def upsert_vectors(
        index: Pinecone,
        dataset: DataFrame,
        embeddings: np.ndarray,
        batch_size: int = 128
):
    """
    Upsert vectors to a pinecone index
    Args:
        index: The pinecone index object
        embeddings: The embeddings to upsert
        dataset: The dataset containing the metadata
        batch_size: The batch size to use for upserting
    Returns:
        An updated pinecone index
    """
    print("Upserting the embeddings to the Pinecone index...")
    
    # Get all column names except 'embedding' for metadata
    metadata_fields = [col for col in dataset.columns if col != "embedding"]

    # Generate unique IDs for each row
    ids = [str(i) for i in range(shape[0])]

    # Build metadata dict for each row
    meta = []
    for _, row in dataset.iterrows():
        entry = {col: row[col] for col in metadata_fields}
        meta.append(entry)  # Extract full metadata

    # Create list of (id, vector, metadata) tuples for upserting
    to_upsert = list(zip(ids, embeddings, meta))

    # Upsert in batches
    for i in tqdm(range(0, len(to_upsert), batch_size)):
        i_end = min(i + batch_size, len(to_upsert))
        index.upsert(vectors=to_upsert[i:i_end])

    print("Upserting complete!")
    return index


In [144]:
# Upsert the embeddings to the Pinecone index
index = pc.Index(INDEX_NAME)
index_upserted = upsert_vectors(index,messages_sent,embeddings)

Upserting the embeddings to the Pinecone index...


100%|██████████| 3/3 [00:11<00:00,  3.78s/it]

Upserting complete!


In [145]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 516}},
 'total_vector_count': 516,
 'vector_type': 'dense'}

## creating propmt

In [314]:
def augment_prompt(
        query: str,
        user_style: str,
        context: str,
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        index=None,
) -> str:

    results = [float(val) for val in list(model.encode(query))]

    # get top 10 results from knowledge base
    query_results = index.query(
        vector=results,
        top_k=5,
        include_values=True,
        include_metadata=True
    )['matches']
    text_matches = [match['metadata']['doc_text'] for match in query_results]


    # get the text from the results
    answers = "\n\n".join(text_matches)
    

    # feed into an augmented prompt
    improved_prompt = f"""
    
      You write WhatsApp replies *exactly as the user would*.

      Your job: given a new incoming message, write the reply the user is most likely to send.

      You are given:
      1) **query** – the new incoming message you must answer.
      2) **similar_past_answers** – real replies the user wrote in the past to similar messages.  
         Use them for tone, vibe, typical phrasing, emojis, and attitude.
      3) **user_style_examples** – random messages the user wrote in other chats.  
         Use them to mimic writing style, vocabulary, length, energy level, and emoji habits.
      4) **recent_context** – the recent messages in this same chat (both sides).  
         Your reply must fit naturally after this context.

      ### Rules:
      - Write the reply **as the user**, in first person.
      - Match the **language**, **tone**, and **emotion** of the conversation.
      - Keep it natural for WhatsApp: short to medium length, can include emojis.
      - If the query contains multiple questions – answer all.
      - If necessary info is missing – ask a short clarifying question.
      - **Never** mention examples, past messages, embeddings, or that you're an AI.
      - **Only output the final WhatsApp-style reply. No explanations.**

      ---

      ### query:
      {query}

      ### similar_past_answers for simillar queries:
      {answers}

      ### user_style_examples:
      {user_style}

      ### recent_context:
      {context}
      """


    return improved_prompt, answers

## Load our Fine Tuned TinyLLama

In [184]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def load_model(model_path: str):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path).to(DEVICE)
    model.eval()
    return model, tokenizer

def llama_generate(prompt, model, tokenizer, max_new_tokens=80):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.8,
            top_p=0.9,
            do_sample=True
        )

    input_ids = inputs["input_ids"][0]
    generated_ids = outputs[0][len(input_ids):]

    answer = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return answer.strip()

In [55]:
FT_MODEL_PATH = "/Users/ransela/merged"
fine_tuned_model, fine_tuned_tokenizer = load_model(FT_MODEL_PATH)

## Generate response using augmented prompt from Fine Tuned TinyLLama

In [ ]:
def generate_augmented_answer(query, user_style, context, model_to_gen, tokenizer_to_gen, model=model_emb, index=index):
  augmented_prompt, source_knowledge, query_results = augment_prompt(query, user_style, context,model=model_emb,index=index)
  answer = llama_generate(augmented_prompt, model_to_gen, tokenizer_to_gen)
  return answer, source_knowledge, query_results

In [311]:
# build context and user style strings
context = build_context(kb_df, conv_id='chat:u_barbara_u_maayan', k=10)
user_style = build_user_style(kb_df, user_id='u_barbara', k=10)


query = "i need help with my students, did you taught them already the embeddings ppt?"

In [ ]:
# Call the full RAG + FT generation pipeline
answer, retrieved_docs = generate_augmented_answer(
    query=query,
    user_style=user_style,
    context=context,
    model_to_gen=fine_tuned_model,
    tokenizer_to_gen=fine_tuned_tokenizer,        
    model=model_emb,          
    index=index        # Pinecone index
)

# 4) Print results
print("====== GENERATED ANSWER ======")
print(answer)

# print("\n====== RETRIEVED DOCUMENTS ======")
# for d in retrieved_docs:
#     print("-", d)


for text_match in query_results:
    print("----" ,text_match)

In [ ]:

augmented_prompt, source_knowledge = augment_prompt(query, user_style, context,model=model_emb,index=index)
co = cohere.Client(api_key=COHERE_API_KEY)
response = co.chat(
        model='command-a-03-2025',
        message=augmented_prompt,
    )
response.text

### DEBUG FOR LLAMA AND FINE_TUNED LLAMA

In [189]:
test_prompt = (
    """
    You are a friendly WhatsApp user named Barbara. Write a reply to the message: 'how are you?' \n
    Your response MUST be 1–2 short, casual sentences, starting immediately. \n
    Provide the reply text ONLY, without any instruction, context, quotes, or labels. \n 
    """
)
# The expected output should now be something clean like:
# "Hey! I'm good, just finishing up a lab project. How about you?"



test generator with fine tuned llama

In [167]:
print(llama_generate(test_prompt, fine_tuned_model, fine_tuned_tokenizer, max_new_tokens=40))

How are you? I’m doing great. You? Same here. I hope you’re doing good too. We are. That sounds great. Thanks. You’re welcome. You'


test generator with basic tiny llama model

In [ ]:
BASE_MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID).to(DEVICE)
base_model.eval()

In [190]:
print(llama_generate(test_prompt, base_model, base_tokenizer, max_new_tokens=40))

Sample 2: 
    Message: 'How are you today, Barbara? Looking forward to catching up with you soon.' 
    Reply: 'Hey, Barbara
